In [19]:
from scipy.stats import binom
import math
from scipy.stats import multinomial
import numpy as np

In [2]:
class Sequence_t_old:
    def __init__(self, L, n, p, k):
        self.L = L
        self.n = n
        self.p = p
        self.k = k
        self.read_probability = [binom.pmf(i,self.k,self.n/self.L) for i in range(self.k+1)]
    def prob_half(self, n):
        if n==0:
            return self.read_probability[n]*0.75
        prob = binom.sf(n/2, n, self.p)
        if n % 2 == 0:
            prob += binom.pmf(n/2, n, self.p)*0.5
        return prob*self.read_probability[n]
    def sum_err(self):
        return self.L*sum([self.prob_half(i) for i in range(self.k+1)])

In [3]:
class Sequence_t:
    def __init__(self, L, n, p, k):
        self.L = L
        self.n = n
        self.p = p
        self.k = k
        self.read_probability = [binom.pmf(i,self.k,self.n/self.L) for i in range(self.k+1)]
    def prob_bad_nucleotide(self, n, p):
        bad_prob = 0  
        bad_prob += binom.sf(math.floor(n/2), n, p)
        if n % 2 == 0:
            bad_prob += binom.pmf(n/2, n, p)*0.5
        return bad_prob
    def sum_err(self, prob_bad_nucleotide):
        one_nucl_err = self.read_probability[0]*0.75
        for k in range(1, self.k+1):
            bad_prob = prob_bad_nucleotide(k, self.p)
            one_nucl_err += bad_prob*self.read_probability[k]    
        return self.L*one_nucl_err

In [21]:
def prob_bad_nucleotide(k, p):
    sequencer = multinomial(k, [1-p, (p)/3, (p)/3, (p)/3])
    fault_tmp = 0
    x = range(0, k+1)
    for e in range(0, k+1):
        for i in range(0, k+1):
            for s in range(0, k+1):
                for r in range(0, k+1):
                    if (i + s + r + e == k):
                        if (p/3)**(s+r+i) > 10e-20:
                            if ((i >= s and i >= r and i > e) or \
                            (s >= i and s >= r and s > e) or (r >= i and r >= s and r > e)):
                                fault_tmp += sequencer.pmf([e, i, s, r])
                            elif((i == e and i > r and i > s) or \
                            (s == e and s > i and s > r) or (r == e and r > s and r > i)):
                                fault_tmp += sequencer.pmf([e, i, s, r]) * 0.5
                            elif ((i == e and s == e and i > r) or \
                            (s == e and r == e and s > i) or (r == e and i == e and r > s)):
                                fault_tmp += sequencer.pmf([e, i, s, r]) * 2 / 3.0
                            elif (i == e and s == e and r == e):
                                fault_tmp += sequencer.pmf([e, i, s, r]) * 0.75
    return fault_tmp

In [22]:
Sequence = Sequence_t(10, 3, 0.050, 4)
Sequence.read_probability

[0.24010000000000004,
 0.41160000000000013,
 0.2646,
 0.07559999999999996,
 0.008099999999999996]

In [23]:
sequencer = multinomial(10, [0.050, (1-0.050)/3, (1-0.050)/3, (1-0.050)/3])
print(sequencer.pmf([4, 3, 2, 1]))
print(sequencer.pmf([4, 3, 3, 0])*3)
print(binom.pmf(4, 10, 0.050))

7.940807460455251e-05
7.940807460455242e-05
0.0009648081064453133


In [24]:
Sequence.sum_err(prob_bad_nucleotide)

2.1433536

In [14]:
Sequence.sum_err(Sequence.prob_bad_nucleotide)

2.1449182500000004

In [25]:
sequences_f = open('2.txt')
sequences = [list(map(lambda x: float(x), line.rstrip().split(' '))) for line in sequences_f]
del sequences[0]
sequences

[[3.0, 2.0, 0.0, 1.0],
 [3.0, 2.0, 0.0, 2.0],
 [15.0, 6.0, 0.005, 10.0],
 [15.0, 3.0, 0.027, 18.0],
 [31.0, 5.0, 0.054, 43.0],
 [81.0, 17.0, 0.085, 80.0],
 [210.0, 11.0, 0.034, 92.0],
 [321.0, 60.0, 0.016, 99.0],
 [711.0, 36.0, 0.088, 120.0],
 [1693.0, 62.0, 0.001, 150.0]]

In [26]:
results = []
for sequence in sequences:
    Sequence = Sequence_t(int(sequence[0]), int(sequence[1]), sequence[2], int(sequence[3]))
    results.append(Sequence.sum_err(prob_bad_nucleotide))
    print(results[-1])

0.75
0.25000000000000006
0.08040206039130798
0.31372053562685387
0.07046934731346188
0.0003522655019140977
2.1591633796667775
4.546154697592807e-06
6.663244575950779
4.850696809279941


In [27]:
output = open('sequence_output_2.txt', 'w')
for result in results:
    output.write(str(result) + '\n')
output.close()

In [7]:
sequences_f = open('1.txt')
sequences = [list(map(lambda x: float(x), line.rstrip().split(' '))) for line in sequences_f]
del sequences[0]
sequences

[[3.0, 2.0, 0.0, 1.0],
 [3.0, 2.0, 0.0, 2.0],
 [15.0, 6.0, 0.005, 10.0],
 [15.0, 3.0, 0.027, 18.0],
 [31.0, 5.0, 0.054, 43.0],
 [81.0, 17.0, 0.085, 80.0],
 [210.0, 11.0, 0.034, 156.0],
 [4341.0, 23.0, 0.016, 3681.0],
 [8573.0, 31.0, 0.088, 100.0],
 [8093.0, 43.0, 0.007, 2511.0]]

In [8]:
results = []
for sequence in sequences:
    Sequence = Sequence_t(int(sequence[0]), int(sequence[1]), sequence[2], int(sequence[3]))
    results.append(Sequence.sum_err(Sequence.prob_bad_nucleotide))
    print(results[-1])

0.75
0.25000000000000006
0.08064580253365887
0.32026643311433217
0.09893372076920791
0.0037191599387386715
0.180537122319785
0.00022425750024889762
4701.699764067475
0.022231836970356054


In [9]:
results

[0.75,
 0.25000000000000006,
 0.08064580253365887,
 0.32026643311433217,
 0.09893372076920791,
 0.0037191599387386715,
 0.180537122319785,
 0.00022425750024889762,
 4701.699764067475,
 0.022231836970356054]

In [10]:
output = open('sequence_output.txt', 'w')
for result in results:
    output.write(str(result) + '\n')
output.close()

In [33]:
math.floor(2.9)

2

In [71]:
def compositions(n,k):
  if n < 0 or k < 0:
    return
  elif k == 0:
    # the empty sum, by convention, is zero, so only return something if
    # n is zero
    if n == 0:
      yield []
    return
  elif k == 1:
    yield [n]
    return
  else:
    for i in range(0,n+1):
      for comp in compositions(n-i,k-1):
        yield [i] + comp

In [79]:
def bad_nucleotides(n, k):
    i = 0
    for comp in compositions(n-k, 3):
        if comp[0] >= k or comp[1] >= k or comp[2] >= k:
            if (comp[0] == k and comp[1] < k and comp[2] < k) or \
            (comp[1] == k and comp[0] < k and comp[2] < k) or \
            (comp[2] == k and comp[0] < k and comp[1] < k):
                i += 
    return i

In [76]:
def eq1_nucleotides(n, k):
    i = 0
    for comp in compositions(n-k, 3):
        if (comp[0] == k and comp[1] < k and comp[2] < k) or \
        (comp[1] == k and comp[0] < k and comp[2] < k) or \
        (comp[2] == k and comp[0] < k and comp[1] < k):
            i += 1
    return i

In [77]:
def eq2_nucleotides(n, k):
    i = 0
    for comp in compositions(n-k, 3):
        if (comp[0] == k and comp[1] == k and comp[2] < k) or \
        (comp[1] == k and comp[2] == k and comp[0] < k) or \
        (comp[0] == k and comp[2] == k and comp[1] < k):
            i += 1
    return i

In [80]:
def eq3_nucleotides(n, k):
    i = 0
    for comp in compositions(n-k, 3):
        if (comp[0] == k and comp[1] == k and comp[2] == k):
            i += 1
    return i

In [ ]:
def prob_bad_nucleotide_2(k, p):
    sequencer = multinomial(k, [p, (1-p)/3, (1-p)/3, (1-p)/3])
    k1_bound = math.floor(k/2)
    if k%2 == 0:
        eq_prob+= sequencer.pmf([k1_bound, k1_bound, 0, 0])*num_eq(k, k1_bound)
        k1_bound = k1_bound - 1 
    for k1 in range(0, k1_bound+1):
        eq_prob += sequencer.pmf([k1, k1, k - 2*k1, 0])*num_eq(k, k1)
        bad_prob += sequencer.pmf([k1, k1 + 1, k - 2*k1 - 1, 0])*num_bad(k, k1)
    return eq_prob, bad_prob

In [13]:
def bad_nucleotides(n):
    for comp in compositions(n,4):
        if comp[0] < comp[1] or comp[0] < comp[2] or comp[0] < comp[3]:
            yield comp

In [14]:
def eq_nucleotides(n):
    for comp in compositions(n,4):
        if (comp[0] == comp[1] and comp[0] >= comp[2] and comp[0] >= comp[3]) or \
        (comp[0] == comp[2] and comp[0] >= comp[1] and comp[0] >= comp[3]) or \
        (comp[0] == comp[3] and comp[0] >= comp[1] and comp[0] >= comp[2]):
            yield comp

In [16]:
generator = eq_nucleotides(100)
eq_nucl_arr = []
for i in generator:
    eq_nucl_arr.append(i) 

In [105]:
import numpy as np
x = np.arange(0, 10, 1)
x1, x2, x3, x4 = np.meshgrid(x,x,x,x)

In [107]:
from scipy.stats import multinomial
def prob_bad_nucleotide(k, p):
    eq_prob = 0
    bad_prob = 0
    sequencer = multinomial(k, [1-p, (p)/3, (p)/3, (p)/3])
    k1_bound = math.floor(k/2)
    if k%2 == 0:
        eq_prob+= sequencer.pmf([k1_bound, k1_bound, 0, 0])*num_eq(k, k1_bound)
        k1_bound = k1_bound - 1 
    for k1 in range(0, k1_bound+1):
        eq_prob += sequencer.pmf([k1, k1, k - 2*k1, 0])*num_eq(k, k1)
        bad_prob += sequencer.pmf([k1, k1 + 1, k - 2*k1 - 1, 0])*num_bad(k, k1)
    return eq_prob*0.75, bad_prob

array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        ...,

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
    

In [ ]:
def num_eq(k, k1):
    return 3*(k - 2*k1 +1)
def num_bad(k, k1):
    return 3*((k-2*k1+1)*(k-2*k1))/2